# 1. Extract data from Source Files 

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# 2. Transforming and Loading of the source data into target Tables for query

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [14]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_ao 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_ao')
except Exception as e:
    print(e)


# Query 1 session_library Data

## Create session_table table

In [16]:
query = "CREATE TABLE IF NOT EXISTS session_table "
query = query + """(

session_id int, 
item_inSession int,
artist_name text, 
song_title text, 
song_length float, 
PRIMARY KEY (session_id, item_inSession)

)"""

try:
    session.execute(query)
except Exception as e:
    print(e)                    

## Insert session_table data

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_table (artist_name, song_title, song_length, session_id, item_inSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

## Validate session_table data

### 1. TEST -> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [18]:
query = "SELECT artist_name, song_title, song_length FROM session_table WHERE session_id=338 AND item_inSession=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2 itemInSession_table Data

## Create itemInSession_table Table

In [19]:
query = "CREATE TABLE IF NOT EXISTS itemInSession_table "
query = query + """(

user_id int,
session_id int,
item_inSession int,
artist_name text, 
song_title text, 
user_name text,
PRIMARY KEY (user_id, session_id)

)"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

## Insert itemInSession_table data

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:              
        query = "INSERT INTO itemInSession_table (artist_name, song_title, user_name, user_id, session_id, item_insession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"     
        session.execute(query, (line[0], line[9], (line[1] + ' ' + line[4]), int(line[10]), int(line[8]), int(line[3]) ))

## Validate itemInSession_library data

### 2. TEST -> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [23]:
query = """
SELECT 
    artist_name, 
    song_title, 
    user_name 
FROM itemInSession_table
WHERE user_id=10 AND session_id=182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name + ', ' + row.song_title + ', ' +  row.user_name)

Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie Cruz


# Query 3 userName_table Data

## Create userName_table Table

In [24]:
query = "CREATE TABLE IF NOT EXISTS userName_table "
query = query + """(

song_title text, 
user_id int,
user_name text,
PRIMARY KEY (song_title, user_id)

)"""
try:
    session.execute(query)
except Exception as e:
    print(e)

## Insert userName_table data

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:              
        query = "INSERT INTO userName_table (song_title, user_name, user_id)"
        query = query + " VALUES (%s, %s, %s)"     
        session.execute(query, (line[9], (line[1] + ' ' + line[4]), int(line[10]) ))

## Validate userName_table data

### 3. TEST -> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [26]:
query = """
SELECT
    user_name 
FROM userName_table
WHERE song_title='All Hands Against His Own' 
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [11]:
query = "DROP TABLE IF EXISTS userName_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
query = "DROP TABLE IF EXISTS itemInSession_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
query = "DROP TABLE IF EXISTS session_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()